# TensorFlow 2.0 Beta Environment Testing

###  *bigDataSpark Forum @ 2019/08/05*

--------------------------------------
+ ###  First, install `Anaconda 2019.07 for Windows` with `Python 3.7 version` https://www.anaconda.com/distribution/

+ ### Then, run `TensorFlow 2.0 Beta (for CPU)` Setup on `Anaconda Prompt` in *Win10* :
>###           1.  `conda create -n tf2`
>###           2.  `pip install tensorflow=2.0.0-beta1`

### [ Reference ]:
+ TensorFlow.org, **"Install TensorFlow with pip"** https://www.tensorflow.org/install/pip
+ 海萨, **"Anaconda 安装tensorflow 2.0 报错解决办法"** https://zhuanlan.zhihu.com/p/62031082
+ khoa, **"Install TensorFlow-gpu 2.0 Beta on Anaconda for Windows 10/Ubuntu"** https://medium.com/@shaolinkhoa/install-tensorflow-gpu-2-0-alpha-on-anaconda-for-windows-10-ubuntu-ced099010b21
+ TensorFlow.org, **"Get Started with TensorFlow"** https://www.tensorflow.org/tutorials/#get-started-with-tensorflow

## [ Content ]
- [1. Testing TF 2.0](#TF2)
- [2. How to run TensorFlow 1.x code on TF 2.0](#RunTF1)

<a id='TF2'></a>
## 1. Testing TF 2.0

In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


In [2]:
# ---------------------------------------------
# The following code is adopted from 
# Tutorial document of TensorFlow.org 
# for testing TensorFlow 2.0 setup: 
#
# "Get Started with TensorFlow" 
#  https://www.tensorflow.org/tutorials/#get-started-with-tensorflow
# ---------------------------------------------

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

W0805 20:18:42.118590 10288 deprecation.py:323] From C:\Users\USER\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.2209 - accuracy: 0.9351
Epoch 2/5
60000/60000 [==============================] - 10s 170us/sample - loss: 0.0982 - accuracy: 0.9694
Epoch 3/5
60000/60000 [==============================] - 10s 173us/sample - loss: 0.0708 - accuracy: 0.9779
Epoch 4/5
60000/60000 [==============================] - 10s 174us/sample - loss: 0.0528 - accuracy: 0.9833
Epoch 5/5
10000/10000 [==============================] - 1s 69us/sample - loss: 0.0702 - accuracy: 0.9788


[0.0702000554678496, 0.9788]

------------------------------
<a id='RunTF1'></a>
## 2. How to run TensorFlow 1.x code on TF 2.0
+ ### It is still possible to run 1.X code, unmodified (except for contrib), in TensorFlow 2.0:

> ###  **`import tensorflow.compat.v1 as tf`**  
> ###  **`tf.disable_v2_behavior()`**

In [19]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

print(tf.__version__)

2.0.0-beta1


---------------------------------------------
### The following code is adopted for testing TensorFlow 2.0 setup from the reference below: 

+ Tom Hope, Yehezkel S. Resheff, and Itay Lieder, "**Learning TensorFlow : A Guide to Building Deep Learning Systems**," Chapter 2 & 4, O'Reilly, 2017. https://goo.gl/iEmehh
+ Download the code from GitHub : https://github.com/gigwegbe/Learning-TensorFlow
---------------------------------------------

### Loading the MNIST dataset (from TensorFlow 2.0)

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
import numpy as np

x_train = np.array([x_train[i].flatten() for i in range(len(x_train))])
x_train.shape

(60000, 784)

In [6]:
x_test = np.array([x_test[i].flatten() for i in range(len(x_test))])
x_test.shape

(10000, 784)

In [7]:
y_train[0], y_test[0]

(5, 7)

In [8]:
def one_hot(vec, vals=10):
    n = len(vec)
    out = np.zeros((n, vals))
    out[range(n), vec] = 1
    return out

In [9]:
y_train = one_hot(y_train)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [10]:
y_test = one_hot(y_test)
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

### Building a computation graph

In [11]:
# Each Input Image, X, with 28*28 (= 784) pixels
X = tf.placeholder(tf.float32, [None, 784])

# y_true : the training labeled dataset 
y_true = tf.placeholder(tf.float32,[None, 10])

In [12]:
# Initializing Weights & Biases for Nodes in All Hidden Layers 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) 
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape) 
    return tf.Variable(initial)

In [13]:
# Building a Fully-Connected Deep Network
def full_layer(inputs, size):
    in_size = int(inputs.get_shape()[1]) 
    W = weight_variable([in_size, size]) 
    b = bias_variable([size])
    return tf.add(tf.matmul(inputs, W), b)

In [14]:
keep_prob = tf.placeholder(tf.float32)  

# < Hidden Layer 1 >
layer_1_drop = tf.nn.dropout(X, keep_prob=keep_prob)
#   Activation Function : ReLU
layer_1_Outputs = tf.nn.relu(full_layer(layer_1_drop, 256))

# < Hidden Layer 2 >
layer_2_drop = tf.nn.dropout(layer_1_Outputs, keep_prob=keep_prob)
#   Activation Function : ReLU
layer_2_Outputs = tf.nn.relu(full_layer(layer_2_drop, 128))  

# < Output Layer >
output_drop = tf.nn.dropout(layer_2_Outputs, keep_prob=keep_prob)
# Without Activation Function
y_pred = full_layer(output_drop, 10)  

W0805 20:19:38.125968 10288 deprecation.py:506] From <ipython-input-14-931f684597d4>:4: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

W0805 20:19:38.264666 10288 deprecation.py:323] From <ipython-input-15-315e39f82a5d>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Launching the graph

In [16]:
def next_batch(i, images, labels, batch_size):
    i_start = (i * batch_size) % len(images)
    x, y = images[i_start : i_start+batch_size], labels[i_start : i_start+batch_size]
    return x, y

In [17]:
NUM_STEPS = 8000
MINIBATCH_SIZE = 100
Display_Step = 1000

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(NUM_STEPS):
        batch_xs, batch_ys = next_batch(i, x_train, y_train, MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict ={X: batch_xs, 
                                      y_true: batch_ys,
                                      keep_prob: 0.5})
        
        if (i+1) % Display_Step == 0:
            # Calculate batch loss and accuracy
            loss_temp, accu_temp = sess.run([cross_entropy, accuracy], 
                                            feed_dict={X: batch_xs, 
                                                       y_true: batch_ys,
                                                       keep_prob: 1.0})
            print("Step " + str(i+1).rjust(4) + \
                  " : Loss = " + "{:.4f}".format(loss_temp) + \
                  ", Accuracy = " + "{:.3f}".format(accu_temp))

    print("\n Computing the test accuracy ... ", end = " ")
    
    ##  ------------------------------------------------------------------
    ##  Split the test procedure into 10 blocks of 1,000 images each. 
    ##  Doing this is important mostly for much larger datasets.
    ##  ------------------------------------------------------------------
    ##  mnist.test.images.shape : (10000, 784)
    X_test = x_test.reshape(10, 1000, 784) 
    ##  mnist.test.labels.shape : (10000, 10)
    Y_test = y_test.reshape(10, 1000, 10)   
    
    test_loss = np.mean([sess.run(cross_entropy,
                                  feed_dict={X: X_test[i], 
                                             y_true: Y_test[i], 
                                             keep_prob: 1.0}) 
                                  for i in range(10)])
    test_accu = np.mean([sess.run(accuracy,
                                  feed_dict={X: X_test[i], 
                                             y_true: Y_test[i], 
                                             keep_prob: 1.0}) 
                                  for i in range(10)])
    print("\n [ Test  Accuracy ] : {}".format(test_accu) +
      "\n [ Test Loss Score ] : {}".format(test_loss))

Step 1000 : Loss = 0.2291, Accuracy = 0.930
Step 2000 : Loss = 0.1149, Accuracy = 0.980
Step 3000 : Loss = 0.1018, Accuracy = 0.990
Step 4000 : Loss = 0.0792, Accuracy = 0.980
Step 5000 : Loss = 0.0845, Accuracy = 0.970
Step 6000 : Loss = 0.1002, Accuracy = 0.990
Step 7000 : Loss = 0.0499, Accuracy = 0.990
Step 8000 : Loss = 0.1032, Accuracy = 0.970

 Computing the test accuracy ...  
 [ Test  Accuracy ] : 0.9614999890327454
 [ Test Loss Score ] : 0.1306430548429489
